## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embeddings, load one from gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2024-09-05 08:30:17--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2024-09-05 08:30:18--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc22c2b5b1bcdaad85d79197002a.dl.dropboxusercontent.com/cd/0/inline/CZ_JGjNWvDv2ZoC-LY5KT6LkLjyA33xX7EUAO5RCufA45oscqxCxX2bipQFKo2rXb_0jX6SeWcE4RJPSP9nq0ZYNRn54F3q9XWp3NfWn41kOljktj92gTOJsj_hu9kRgAsg/file?dl=1# [following]
--2024-09-05 08:30:18--  https://uc22c2b5b1bcdaad85d79197002a.dl.dropboxusercontent.com/cd/0/inline/CZ_JGjNWvDv

In [2]:
import numpy as np

with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an NLP task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many NLP tasks like tokenization, stemming or part-of-speech tagging.

In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [4]:
# TASK: lowercase everything and extract tokens with tokenizer.
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(x.lower()) for x in data]

In [5]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [6]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings.

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [7]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok,
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [8]:
# now you can get word vectors !
model.get_vector('anything')

array([-3.3679752 ,  0.6567592 ,  0.99564505,  2.2460866 ,  1.8360564 ,
        1.2197566 ,  2.6907449 , -2.142263  , -1.1902682 ,  2.527335  ,
       -3.535046  ,  1.2985045 ,  4.3244348 ,  1.2549589 ,  2.4423034 ,
        0.3658778 , -0.82430905, -2.179155  ,  1.6219622 , -2.2823944 ,
       -3.1530113 , -1.0092552 , -2.6050847 , -1.9739813 ,  0.5438141 ,
       -2.1987457 , -0.22743252,  0.2738007 ,  1.1069769 ,  1.072867  ,
       -1.5703373 ,  0.14810899], dtype=float32)

In [9]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9460483193397522),
 ('cheese', 0.9394707083702087),
 ('fruit', 0.9359089136123657),
 ('banana', 0.9328295588493347),
 ('sauce', 0.93275386095047),
 ('butter', 0.9141667485237122),
 ('potato', 0.9101894497871399),
 ('chicken', 0.9086733460426331),
 ('vodka', 0.9066662192344666),
 ('wine', 0.9065707325935364)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts.

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [10]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [11]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.4922019839286804),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [12]:
words = model.index_to_key[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [15]:
print(len(words))

1000


In [24]:
# for each word, compute it's vector with model
word_vectors = np.array([model[x] for x in words])

In [25]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [33]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca = PCA(n_components=2, whiten=True)

word_vectors_pca_test = pca.fit_transform(word_vectors)

scaler = StandardScaler()
word_vectors_pca = scaler.fit_transform(word_vectors_pca_test)

In [34]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [35]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [36]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1102', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [39]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca
tsne = TSNE(n_components=2)
word_vectors_tsne_test = tsne.fit_transform(word_vectors)

# scaler = StandardScaler()
# word_tsne = scaler.fit_transform(word_vectors_tsne_test)

In [40]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1200', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [41]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating its word embeddings.
    """
    # 1. Lowercase the phrase
    phrase_lower = phrase.lower()

    # 2. Tokenize the phrase
    phrase_token = tokenizer.tokenize(phrase_lower)

    # 3. Initialize a zero vector
    vector = np.zeros(model.vector_size, dtype='float32')

    # 4. Count valid words
    count = 0

    # 5. Aggregate word vectors
    for word in phrase_token:
        if word in model:
            vector += model[word]
            count += 1

    # 6. Average the vectors (if there are any valid words)
    if count > 0:
        vector /= count

    return vector

In [42]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [45]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(x) for x in chosen_phrases])

In [46]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [49]:
# Применение t-SNE для понижения размерности до 2D
tsne = TSNE(n_components=2, perplexity=30, n_iter=300, random_state=42)
phrase_vectors_2d = tsne.fit_transform(phrase_vectors)

# Нормализация
scaler = StandardScaler()
phrase_vectors_2d = scaler.fit_transform(phrase_vectors_2d)


In [50]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1298', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [51]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [87]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def find_nearest(query, k=10):
    """
    Given a text line (query), return k most similar lines from data, sorted from most to least similar.
    Similarity should be measured as cosine between query and line embedding vectors.
    """
    # Получаем вектор для строки запроса и изменяем форму
    query_vector = get_phrase_embedding(query).reshape(1, -1)

    # Вычисляем косинусное сходство между вектором запроса и всеми векторами данных
    sim = cosine_similarity(data_vectors, query_vector).flatten()

    # Получаем индексы k наиболее похожих строк
    top_k_indices = np.argsort(-sim)[:k]

    # Формируем список строк с результатами
    top_k_lines = [data[idx].strip() for idx in top_k_indices]

    return top_k_lines


In [88]:
results = find_nearest(query="How do i enter the matrix?", k=10)
print(results)

# print(''.join(results))

# assert len(results) == 10 and isinstance(results[0], str)
# assert results[0] == 'How do I get to the dark web?\n'
# assert results[3] == 'What can I do to save the world?\n'

['How do I get to the dark web?', 'What should I do to enter hollywood?', 'How do I use the Greenify app?', 'What can I do to save the world?', 'How do I win this?', 'How do I think out of the box? How do I learn to think out of the box?', 'How do I find the 5th dimension?', 'How do I use the pad in MMA?', 'How do I estimate the competition?', 'What do I do to enter the line of event management?']


In [89]:
find_nearest(query="How does Trump?", k=10)

['What does Donald Trump think about Israel?',
 'What books does Donald Trump like?',
 'What does Donald Trump think of India?',
 'What does India think of Donald Trump?',
 'What does Donald Trump think of China?',
 'What does Donald Trump think about Pakistan?',
 'What companies does Donald Trump own?',
 'What does Dushka Zapata think about Donald Trump?',
 'How does it feel to date Ivanka Trump?',
 'What does salesforce mean?']

In [90]:
find_nearest(query="Why don't i ask a question myself?", k=10)

["Why don't I get a date?",
 "Why do you always answer a question with a question? I don't, or do I?",
 "Why can't I ask a question anonymously?",
 "Why don't I get a girlfriend?",
 "Why don't I have a boyfriend?",
 "I don't have no question?",
 "Why can't I take a joke?",
 "Why don't I ever get a girl?",
 "Can I ask a girl out that I don't know?",
 "Why don't I have a girlfriend?"]

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize `find_nearest` with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.